# Theme of this notebook - how to calculate performance metrics (CV, Sharpe Ratio, Treynor Ratio, Sortino Ratio)
* These are codes for calculating performance metrics of a portfolio
* Let's practice throughout solving sample questions.

# Question
Using monthly data for the period 2018M1 to 2022M12, compute and compare the following finance ratios for APPL and TSLA:<br>
1. Coefficient of Variation (CV)
2. Sharpe Ratio
3. Treynor Ratio
4. Sharpe Ratio

# 1.Coefficient of Variation (CV)

Basically, I followed the steps below.<br>
1. Choose one of the models (*I adopted one-factor model).<br>
2. Find E($R_a$) = $\hat{\alpha}$ +$\hat{\beta}R_m$ for the time period.<br>
3. Calculate excess returns, its volatility, beta, negative returns, etc.

In [65]:
# First, retrive data from Yahoo Finance
! pip install pandas-datareader

In [66]:
# Import libraries
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf

In [67]:
# Retrive Apple's data from 2018M1 to 2022M12 on a monthly basis
apple = yf.download('AAPL', start='2018-01-01', end='2022-12-31', interval='1mo')
# Retrive Tesla (TSLA)'s data from 2018M1 to 2022M12 on a monthly basis
tesla = yf.download('TSLA',start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [68]:
# Set a variable named returns
returns = pd.DataFrame()

In [69]:
# Calculate each return
returns['AAPL_ret'] = (apple.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()
returns['TSLA_ret'] = (tesla.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()

returns

,AAPL_ret,TSLA_ret
Date,,
2018-01-01,NaN,NaN
2018-02-01,0.063848,-0.031752
2018-03-01,-0.054211,-0.224246
2018-04-01,-0.015019,0.104347
2018-05-01,0.130764,-0.031201
2018-06-01,-0.005599,0.204474
2018-07-01,0.027983,-0.130660
2018-08-01,0.196227,0.011807
2018-09-01,-0.004825,-0.122290


In [70]:
# Calculate mean and volatility(= standard deviation)
mean_std = pd.DataFrame({'Mean': returns.dropna().mean(),
                        'Volatility': returns.dropna().std()})

mean_std

,Mean,Volatility
AAPL_ret,0.024549,0.094050
TSLA_ret,0.048806,0.217348


Let's calculate Coefficient of Variation (CV)!

# <b>Coefficient of Variation (CV) = $\frac{Standard \ Deviation(s)}{Sample \ Mean (\bar{x})}$ </b>

In [71]:
#Calculate CV
cv_aapl = mean_std.iloc[0,1]/mean_std.iloc[0,0]
cv_tsla = mean_std.iloc[1,1]/mean_std.iloc[1,0]
print(f"Coefficient  of Variation of Apple = {cv_aapl}")
print(f"Coefficient  of Variation of Tesla = {cv_tsla}")

Coefficient  of Variation of Apple = 3.8310515044842837
Coefficient  of Variation of Tesla = 4.45327974012503


#2. Sharpe Ratio

* In order to calculate Sharpe Ratio, I adopted ten-year treasury (^TNX) as a risk-free rate.

In [72]:
# Retrive 10-year treasury data from 2018M1 to 2022M12 on a monthly basis
risk_free = yf.download('^TNX', start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed


In [73]:
# Set monthly returns of 10-year treasury
returns['risk_free_ret'] = (risk_free.loc['2018-01-01':'2022-12-31', 'Adj Close']) / 100
returns

,AAPL_ret,TSLA_ret,risk_free_ret
Date,,,
2018-01-01,NaN,NaN,0.02720
2018-02-01,0.063848,-0.031752,0.02868
2018-03-01,-0.054211,-0.224246,0.02741
2018-04-01,-0.015019,0.104347,0.02936
2018-05-01,0.130764,-0.031201,0.02822
2018-06-01,-0.005599,0.204474,0.02849
2018-07-01,0.027983,-0.130660,0.02964
2018-08-01,0.196227,0.011807,0.02853
2018-09-01,-0.004825,-0.122290,0.03056


In [74]:
# Calculate excess returns of AAPL and TSLA
# Excess return = Return on each security - Risk free rate
returns['Excess_return_AAPL'] = (returns.iloc[:,0]-returns.iloc[:,2])
returns['Excess_return_TSLA'] = (returns.iloc[:,1]-returns.iloc[:,2])
returns

,AAPL_ret,TSLA_ret,risk_free_ret,Excess_return_AAPL,Excess_return_TSLA
Date,,,,,
2018-01-01,NaN,NaN,0.02720,NaN,NaN
2018-02-01,0.063848,-0.031752,0.02868,0.035168,-0.060432
2018-03-01,-0.054211,-0.224246,0.02741,-0.081621,-0.251656
2018-04-01,-0.015019,0.104347,0.02936,-0.044379,0.074987
2018-05-01,0.130764,-0.031201,0.02822,0.102544,-0.059421
2018-06-01,-0.005599,0.204474,0.02849,-0.034089,0.175984
2018-07-01,0.027983,-0.130660,0.02964,-0.001657,-0.160300
2018-08-01,0.196227,0.011807,0.02853,0.167697,-0.016723
2018-09-01,-0.004825,-0.122290,0.03056,-0.035385,-0.152850


In [75]:
# Create table to calculate the standard deviation of excess return
mean_std = pd.DataFrame({'Mean': returns.dropna().mean(),
                        'Volatility': returns.dropna().std()})

mean_std

,Mean,Volatility
AAPL_ret,0.024549,0.094050
TSLA_ret,0.048806,0.217348
risk_free_ret,0.020367,0.009355
Excess_return_AAPL,0.004182,0.097189
Excess_return_TSLA,0.028439,0.221591


Let's calculate Sharpe ratio!

# <b>Sharpe Ratio = $\frac{R_p - Rf}{\sigma_p}$</b><br>
$R_p$ = Return of portfolio<br>
$R_f$ = Risk-free rate<br>
$\sigma_p$ = Standard Deviation of the portfolio's excess return<br>

In [76]:
#Calculate Sharpe Ratio
sr_aapl = mean_std.iloc[3,0] / mean_std.iloc[3,1]
sr_tsla = mean_std.iloc[4,0] / mean_std.iloc[4,1]
print(f"Sharpe Ratio of Apple = {sr_aapl}")
print(f"Sharpe Ratio of Tesla = {sr_tsla}")

Sharpe Ratio of Apple = 0.04302840529312029
Sharpe Ratio of Tesla = 0.12833923665254338


#3. Treinor Ratio

* In order to calculate Treynor Ratio, I adopeted S&P500 (^GSPC) as a benchmark.

In [77]:
# Retrive apple's data from 2018M1 to 2022M12 on a monthly basis
apple = yf.download('AAPL', start='2018-01-01', end='2022-12-31', interval='1mo')
# Retrive tesla's data from 2018M1 to 2022M12 on a monthly basis
tesla = yf.download('TSLA', start='2018-01-01', end='2022-12-31', interval='1mo')
# Retrive S&P 500(^GSPC)'s data from 2018M1 to 2022M12 on a monthly basis
sp500 = yf.download('^GSPC',start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [78]:
# Set variables
apple = pd.DataFrame(apple.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()
tesla = pd.DataFrame(tesla.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()
sp500 = pd.DataFrame(sp500.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()

# Set intercept to forecast beta by using statsmodel
sp500 = sm.add_constant(sp500)

In [79]:
# Forecast α and β of Apple by using statsmodel
model_apple = sm.OLS(apple.dropna(),sp500.dropna())
result_apple = model_apple.fit()
result_apple.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.523
Method:                 Least Squares   F-statistic:                     64.63
Date:                Sat, 27 Jan 2024   Prob (F-statistic):           5.91e-11
Time:                        02:09:20   Log-Likelihood:                 78.619
No. Observations:                  59   AIC:                            -153.2
Df Residuals:                      57   BIC:                            -149.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0161      0.009      1.888      0.064      -0.001       0.033
Adj Close      1.2695      0.158      8.040      0.000       0.953       1.586
==============================================================================
Omnibus:                        6.203   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.045   Jarque-Bera (JB):                6.210
Skew:                          -0.459   Prob(JB):                       0.0448
Kurtosis:                       4.298   Cond. No.                         18.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
# Set Apple's beta
apple_beta = result_apple.params[1]
apple_beta

1.2695249019485018

In [81]:
# Forecast α and β of Tesla by using statsmodel
model_tesla = sm.OLS(tesla.dropna(),sp500.dropna())
result_tesla = model_tesla.fit()
result_tesla.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.241
Method:                 Least Squares   F-statistic:                     19.46
Date:                Sat, 27 Jan 2024   Prob (F-statistic):           4.62e-05
Time:                        02:09:20   Log-Likelihood:                 15.500
No. Observations:                  59   AIC:                            -27.00
Df Residuals:                      57   BIC:                            -22.84
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0353      0.025      1.420      0.161      -0.014       0.085
Adj Close      2.0303      0.460      4.411      0.000       1.109       2.952
==============================================================================
Omnibus:                        7.186   Durbin-Watson:                   1.568
Prob(Omnibus):                  0.028   Jarque-Bera (JB):                6.302
Skew:                           0.703   Prob(JB):                       0.0428
Kurtosis:                       3.764   Cond. No.                         18.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
# Set Tesla's beta
tesla_beta = result_tesla.params[1]
tesla_beta

2.0303224870757104

Let's calculate Treynor Ratio!

# <b>Treynor Ratio = $\frac{R_p - Rf}{\beta_p}$</b><br>
$R_p$ = Return of portfolio<br>
$R_f$ = Risk-free rate<br>
$\sigma_p$ = Beta of the portfolion<br>

In [83]:
mean_std

,Mean,Volatility
AAPL_ret,0.024549,0.094050
TSLA_ret,0.048806,0.217348
risk_free_ret,0.020367,0.009355
Excess_return_AAPL,0.004182,0.097189
Excess_return_TSLA,0.028439,0.221591


In [84]:
# Calculate Treynor Ratio
tr_aapl = mean_std.iloc[3,0] / apple_beta
tr_tsla = mean_std.iloc[4,0] / tesla_beta
print(f"Treynor Ratio of Apple = {tr_aapl}")
print(f"Treynor Ratio of Tesla = {tr_tsla}")

Treynor Ratio of Apple = 0.003294062587331205
Treynor Ratio of Tesla = 0.014007022939786928


#4. Sortino Ratio

In [85]:
# Retrive data from 2018M1 to 2022M12 on a monthly basis
apple = yf.download('AAPL', start='2018-01-01', end='2022-12-31', interval='1mo')
tesla = yf.download('TSLA', start='2018-01-01', end='2022-12-31', interval='1mo')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [86]:
# Let's extract negative returns of AAPL and Tesla
apple_returns = pd.DataFrame()
apple_returns['AAPL_ret'] = (apple.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()
negative_return_aapl = apple_returns[apple_returns['AAPL_ret'] <= 0]
negative_return_aapl

,AAPL_ret
Date,
2018-03-01,-0.054211
2018-04-01,-0.015019
2018-06-01,-0.005599
2018-09-01,-0.004825
2018-10-01,-0.030477
2018-11-01,-0.184045
2018-12-01,-0.113616
2019-05-01,-0.127573
2019-08-01,-0.020184


In [87]:
tesla_returns = pd.DataFrame()
tesla_returns['TSLA_ret'] = (tesla.loc['2018-01-01':'2022-12-31', 'Adj Close']).pct_change()
negative_return_tsla = tesla_returns[tesla_returns['TSLA_ret'] <= 0]
negative_return_tsla

,TSLA_ret
Date,
2018-02-01,-0.031752
2018-03-01,-0.224246
2018-05-01,-0.031201
2018-07-01,-0.130660
2018-09-01,-0.122290
2018-12-01,-0.050445
2019-01-01,-0.077464
2019-03-01,-0.125109
2019-04-01,-0.147109


In [88]:
# Calculate negative returns std
negative_returns_std = pd.DataFrame({'AAPL': negative_return_aapl.dropna().std(),
                                    'TSLA': negative_return_tsla.dropna().std()})

negative_returns_std

,AAPL,TSLA
AAPL_ret,0.046179,NaN
TSLA_ret,NaN,0.07489


Let's calculate Sortino Ratio!

# <b>Sortino Ratio = $\frac{R_p - Rf}{\sigma_d}$</b><br>
$R_p$ = Return of portfolio<br>
$R_f$ = Risk-free rate<br>
$\sigma_d$ = Standard Deviation of Negative Returns on the portfolio (downside)<br>

In [89]:
mean_std

,Mean,Volatility
AAPL_ret,0.024549,0.094050
TSLA_ret,0.048806,0.217348
risk_free_ret,0.020367,0.009355
Excess_return_AAPL,0.004182,0.097189
Excess_return_TSLA,0.028439,0.221591


In [90]:
sor_aapl = mean_std.iloc[3,0] / negative_returns_std.iloc[0,0]
sor_tsla = mean_std.iloc[4,0] / negative_returns_std.iloc[1,1]
print(f"Sortino Ratio of Apple = {sor_aapl}")
print(f"Sortino Ratio of Tesla = {sor_tsla}")

Sortino Ratio of Apple = 0.09055865600304328
Sortino Ratio of Tesla = 0.37974015618673485


# Merge the results

In [91]:
results = pd.DataFrame()
results['AAPL'] = cv_aapl,sr_aapl,tr_aapl,sor_aapl
results["TSLA"] = cv_tsla,sr_tsla,tr_tsla,sor_tsla
results.index = ['CV', 'Sharpe_Ratio','Treynor_Ratio','Sortino_Ratio']
results

,AAPL,TSLA
CV,3.831052,4.453280
Sharpe_Ratio,0.043028,0.128339
Treynor_Ratio,0.003294,0.014007
Sortino_Ratio,0.090559,0.379740


# Conclusion
* Python enables us to easily calculate performance metrics of a portfolio.